In [2]:
store -r kb_id

In [2]:
import boto3
import pprint
from botocore.client import Config
import json
import time
from botocore.exceptions import ClientError

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-east-1


In [3]:
def retrieve(query, kbId, numberOfResults=10):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )

In [4]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults:
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [12]:
with open("./data/random-items.csv", "r", encoding="utf-8-sig") as f:
    file = f.read()
items = file.split('\n')

with open("./data/output.csv", "w+") as f:
    f.write("TRUPER,GAFI (input),output\n")
    for i in range(0, 200):
        product = items[i].split(',')
        with open("./data/contexts" + str(i) + ".txt", "r", encoding="utf-8-sig") as g:
            contexts = g.read()

        
        response = retrieve(product[1], kb_id, 100)
        contexts = get_contexts(response['retrievalResults'])
        with open("./data/contexts" + str(i) + ".txt", "w+") as g:
            for art in contexts:
                g.write(art + '\n')
        

        prompt = f"""Human: You are an assistant at a hardware store helping a customer find an  item.  The list of items available at the store is 
        enclosed in <context> tags. The item that the customer is looking for is enclosed in <query> tags. The customer's description of the item
        might not be the same exact description as in your list, but make sure that you give the customer an item of the same size/caliber/measurement
        that they are looking for. If you do not find the exact item that the customer is looking for, respond with "not found". If you find an item
        that is the exact item that the customer is looking for, respond only with the item's description from your list and no explanation
        
        <context>
        {contexts}
        </context>
        
        <query>
        {product[1]}
        </query>
        
        Assistant:
        """

        body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 300})
        modelId = "anthropic.claude-instant-v1"
        accept = "application/json"
        contentType = "application/json"

        response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        responsebody = json.loads(response.get("body").read())
        output = responsebody.get("completion")
        output = output.replace(',', '')
        output = output.replace('\n', '/')
        f.write(product[0] + ',' + product[1] + ',' + output + '\n')

In [9]:
product = "Llave combinada 19mm ll-1219m truper"
response = retrieve(product, kb_id, 40)
contexts = get_contexts(response['retrievalResults'])
pp.pprint(contexts)
print()

prompt = f"""Human: You are an assistant at a hardware store helping a customer find an  item.  The list of items available at the store is 
enclosed in <context> tags. The item that the customer is looking for is enclosed in <query> tags. The customer's description of the item might 
not exactly match the description in your list. Make sure that you give the customer an item of the same size/caliber/measurement that they are 
looking for. If you find an item that is the exact item that the customer is looking for, respond only with the item's description from your list
and no explanation. If you do not find the exact item, respond with a list of similar items and no explanation.

<context>
{contexts}
"Llave combinada 19mm ll-1219m truper" is the same as "Llave combinada extralarga milimetrica 19 x 280 mm"
</context>

<query>
{product}
</query>

Assistant:
"""

body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 300})
modelId = "anthropic.claude-instant-v1"
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
responsebody = json.loads(response.get("body").read())
output = responsebody.get("completion")
print(output)

[ 'Llave combinada de 19mm, granallada',
  'Llave combinada de 11mm, granallada',
  'Llave combinada de 12mm, granallada',
  'Llave combinada, extralarga, milimetrica, 9 x 159 mm',
  'Llave combinada de 13mm, granallada',
  'Llave combinada de 17mm, granallada',
  'Llave combinada de 10mm, granallada',
  'Llave combinada de 18mm, granallada',
  'Llave combinada, extralarga, milimetrica, 23 x 319 mm',
  'Llave combinada, extralarga, milimetrica, 22 x 319 mm',
  'Llave combinada de 14mm, granallada',
  'Llave combinada, extralarga, milimetrica, 24 x 339 mm',
  'Llave combinada de 15mm, granallada',
  'Llave combinada, extralarga, milimetrica, 29 x 394 mm',
  'Llave combinada, extralarga, milimetrica, 11 x 184 mm',
  'Llave combinada, extralarga, milimetrica, 12 x 195 mm',
  'Llave combinada, extralarga, milimetrica, 10 x 172 mm',
  'Llave combinada de 9mm, granallada',
  'Llave combinada, extralarga, milimetrica, 7 x 137 mm',
  'Llave combinada, extralarga, milimetrica, 15 x 232 mm',
  '

In [ ]:
product = items[i].split(',')
response = retrieve(product[1], kb_id, 100)
contexts = get_contexts(response['retrievalResults'])
with open("./data/contexts" + str(i) + ".txt", "w+") as g:
    for art in contexts:
        g.write(art + '\n')

prompt = f"""Human: You are an assistant at a hardware store helping a customer find an  item.  The list of items available at the store is 
enclosed in <context> tags. The item that the customer is looking for is enclosed in <query> tags. The customer's description of the item
might not be the same exact description as in your list, but make sure that you give the customer an item of the same size/caliber/measurement
that they are looking for. If you do not find the exact item that the customer is looking for, respond with "not found". If you find an item
that is the exact item that the customer is looking for, respond only with the item's description from your list and no explanation

<context>
{contexts}
</context>

<query>
{product[1]}
</query>

Assistant:
"""

body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 300})
modelId = "anthropic.claude-instant-v1"
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
responsebody = json.loads(response.get("body").read())
output = responsebody.get("completion")
output = output.replace(',', '')
output = output.replace('\n', '/')
f.write(product[0] + ',' + product[1] + ',' + output + '\n')

In [19]:
product = "Abrazadera reforzada # 4 3/8 - 5/8' bolsa 10 pzas"
query = "Abrazadera #04 (10-16mm) 3/8 - 5/8 c/10pz AB4 Fiero"

prompt = f"""You are going to play the role of an assistant at a hardware store and you are going to help a customer find the item that they are looking for. You asked you co-worker to bring you the item that most resembles the customer's description. Now, you must determine if they brought the correct item or not.

First, you must determine what type of item the customer is looking for based on the description. For example, if the customer asks for "Abrazadera #10 (19-27mm) 3/4 - 1-1/16 c/10pz AB10 Fiero," you know they are looking for an "abrazadera".

If the customer mentions any measurement/caliber/length/gauge, you must also make sure that they are the same. In the past example, you should make sure the item is #10 and 3/4 - 1 1/16 inches, or 19-27 mm.

Lastly, if the customer mentions a count, you must also make sure to pick an item that contains the same count. The past examples mentions "10 pz," so make sure to give a customer an item with 10 pieces.

The following is an example of products that are not equivalent:
"Remaches con espiga 6.4x15.9mm bolsa con 50 pzas" and "Remache diam 3/16 agarre 1/2 c/50pz R68B Fiero"
Notice how in this example, the type of item is the same, but the size is different.

The following is an example of products that are equivalent:
"Rotomartillo 1/2' 550 W Pretul" and "Rotomartillo 1/2 550w ROTO1/2P6 Pretul"

The customer is looking for the item with the following description:
"Abrazadera #04 (10-16mm) 3/8 - 5/8 c/10pz AB4 Fiero"

Your co-worker brought the following item:
"Abrazadera de tubo tipo una 1' bolsa con 10 pzas Volteck"

Are they the same item? Responde with a confidence score that represents how likely it is that the item is the same as the item that the customer is looking for."""

for i in range(4):
    body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 10}})
    modelId = "amazon.titan-text-premier-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    output = responsebody.get("results")[0].get("outputText")
    print(output)

0.1
0.05
0.2
0.05


In [ ]:
prompts = []
with open(f"./promptsnuevos/prompts-titan.txt", "r", encoding="utf-8-sig") as f:
    prompts = f.read().split('--------')

csv = "Prompt,Prueba 1,Prueba 2,Prueba 3\n"
for index, prompt in enumerate(prompts):
    if index % 50 == 0:
        print(index)

    csv += str(index) + ','
    for i in range(3):
        try:
            body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 10}})
            modelId = "amazon.titan-text-premier-v1:0"
            accept = "application/json"
            contentType = "application/json"

            response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
            responsebody = json.loads(response.get("body").read())
            output = responsebody.get("results")[0].get("outputText")
        except ClientError:
            output = "skip"
        csv += output.replace('\n', '') + ','
    csv = csv.removesuffix(',') + '\n'

with open("./promptsnuevos/resultados-titan.csv", "w+") as f:
    f.write(csv)

0
50
100
150
200
250
300
350
400
450


In [10]:
print(csv)

Prompt,Prueba 1,Prueba 2,Prueba 3,Prueba 4
0,0.9,0.8,0.9,0.9
1,0.8,0.95,0.9,0.9
2,0.9,0.95,0.9,0.9
3,0.85,0.8,0.85,0.9
4,0.9,0.9,0.8,0.95
5,0.75,0.85,0.9,0.9
6,0.8,0.8,0.9,0.95
7,0.95,0.9,0.9,0.9
8,0.9,0.95,0.9,0.95
9,0.9,0.8,0.95,0.95
10,0.8,0.8,0.9,0.95
11,0.9,0.9,0.9,0.9
12,0.9,0.9,0.95,0.8
13,0.8,0.95,0.9,0.9
14,0.8,0.8,0.9,0.95
15,0.75,0.6,0.5,0.6
16,0.95,0.95,0.85,0.95
17,0.9,0.9,0.9,0.9
18,0.8,0.9,0.7,0.85
19,0.6,0.9,0.8,0.9
20,0.85,0.95,0.8,0.95
21,0.9,0.95,0.9,0.85
22,0.8,0.75,0.85,0.6
23,0.9,0.9,0.85,0.9
24,0.85,0.7,0.8,0.8
25,0.9,0.8,0.8,0.9
26,0.8,0.5,0.9,0.5
27,


In [13]:
with open("./data/random-items.csv", "r", encoding="utf-8-sig") as f:
    matched_items = f.read().split('\n')
matched_items = list(map(lambda x: x.split(','), matched_items))

csv = "TRUPER (input),GAFI,Output\n"

for i, item in enumerate(matched_items):
    if i % 10 == 0:
        print(i)

    with open(f"./data/claudeprompt{i}.txt", "r", encoding="utf-8-sig") as f:
        prompt = f.read()

    body = json.dumps({"inputText": prompt, "textGenerationConfig": {"maxTokenCount": 100}})
    modelId = "amazon.titan-text-premier-v1:0"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    responsebody = json.loads(response.get("body").read())
    output = responsebody.get("results")[0].get("outputText")
    csv += f"{item[0]},{item[1]},{output}\n"

with open("./data/output-claude.txt", "w+") as f:
    f.write(csv)

0
10


KeyboardInterrupt: 